# Module 5: Data Collection & Cleaning Part 2

 
This module consists of 3 parts.

* Part 1 - Data Sources
* Part 2 - Web Scraping
* Part 3 - Data Preparation

Each part is provided in a separate file. It is recommended that you follow the order of the files.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Module-5:-Data-Collection-&amp;-Cleaning" data-toc-modified-id="Module-5:-Data-Collection-&amp;-Cleaning-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Module 5: Data Collection &amp; Cleaning</a></span><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Learning-Outcomes" data-toc-modified-id="Learning-Outcomes-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Learning Outcomes</a></span></li><li><span><a href="#Readings-and-Resources" data-toc-modified-id="Readings-and-Resources-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Readings and Resources</a></span></li><li><span><a href="#Web-Scraping" data-toc-modified-id="Web-Scraping-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Web Scraping</a></span><ul class="toc-item"><li><span><a href="#HTML-&amp;-XML" data-toc-modified-id="HTML-&amp;-XML-1.4.1"><span class="toc-item-num">1.4.1&nbsp;&nbsp;</span>HTML &amp; XML</a></span></li><li><span><a href="#JSON" data-toc-modified-id="JSON-1.4.2"><span class="toc-item-num">1.4.2&nbsp;&nbsp;</span>JSON</a></span></li></ul></li><li><span><a href="#Exercise-1" data-toc-modified-id="Exercise-1-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Exercise 1</a></span></li><li><span><a href="#Scraping-the-Easy-Way" data-toc-modified-id="Scraping-the-Easy-Way-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Scraping the Easy Way</a></span><ul class="toc-item"><li><span><a href="#Python-Libraries-for-Web-Scraping" data-toc-modified-id="Python-Libraries-for-Web-Scraping-1.6.1"><span class="toc-item-num">1.6.1&nbsp;&nbsp;</span>Python Libraries for Web Scraping</a></span></li><li><span><a href="#Links" data-toc-modified-id="Links-1.6.2"><span class="toc-item-num">1.6.2&nbsp;&nbsp;</span>Links</a></span></li></ul></li></ul></li><li><span><a href="#References" data-toc-modified-id="References-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>References</a></span></li></ul></div>

## Web Scraping

Now that we have an idea of what the data looks like in terms of form, we can begin actually scraping for data. Here we cover JSON, and XML as these formats cover the majority of web sources. As a special case we also include HTML.
Previously, we showed how to download a CSV file from a website and loaded into a usable `DataFrame` object. In this section we focus on doing the same so that you'll be able to fully utilize web resources as data. 

### HTML & XML
Python has many libraries for reading and writing data in the HTML and XML formats. `lxml` (Behnel, 2018) is a python library that has consistently strong performance in parsing very large files.
Lets begin using `lxml` by scraping the HTML. 

**NOTE**: In the following example, we can't use `pandas` to scrape. `pd.read_html(trickySite)` will throw an error.  This is because the `<table>` element isn't used by the site.

In [3]:
# Setup code and importing libraries
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import re as re
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)
pd.options.display.max_rows = 6

In [10]:
import requests
import warnings
warnings.filterwarnings("ignore")

trickySite = 'https://xmarquez.github.io/democracyData/reference/pacl.html'
# To get started, find the URL you want to extract data from, open it with *requests*.
r = requests.get(trickySite, verify=False, )
r

<Response [200]>

Now what? We know we have to extract the data, but wasn't this the entire point of using `pandas`? So that we wouldn't have to do this part?

Unfortunately, this part can get very messy, especially for a web-scraping novice. Instead, we will make a simplifying assumption. ***HTML is well formed XML***. If this is true, then we should be able to manipulate and select items as if they are XML. The easiest way to do this is with * XPath * and * XSLT *.

##### XPath
**XML Path Language (XPath)** is a query language for selecting nodes from XML or to compute values from XML. XPath is simply a way to select groups of elements, attributes, text, etc., based on XML tree structure.  

For example, the XPath value `/html/body/a` would grab all `a` elements directly in the `body` element in a root `html` element. However, if an `a` element is embedded inside another element in `body`, then it will not be retrieved. XPath can be thought of as a query language using a syntax similar to file directories. File directories can be absolute or relative. In our example, the path was absolute. If we wanted all `a` elements at any depth inside `body`, then we would use `/html/body//a` as our XPath.

Generally, XPath is used with XML. Since XML defines other formats, XPath is a very useful technology to know when dealing with any markup language. It serves the exact same purpose as Regular Expressions, except for matching tree structures and branches in XML.

#####  XSLT

So far the use cases we've discussed are as follows. 

* XML is data
* XPath is a way to query XML

**eXtensible Stylesheet Language Transformations (XSLT)** would be best described as a way to transform, merge, join, and generally perform operations with and on XML. The use-case of XSLT is to provide a way to rewrite XML data into other formats. XSLT is a subset of XSL, a stylesheet language. Yet, in order for XSLT to remain general, nothing technology specific can be used. This results in a language specifically designed to be good with expressing transformations concisely for XML. 

**NOTE**: XSLT is itself expressed as an XML data format.

For those paying close attention, notice that XPath is the equivalent to a CSS selector. Both need to find groups of elements and then apply their respective snippets of stylesheet code onto them.

Consider the following benefits of these technologies so far.

* XML data is guaranteed to be cleanly parsed. 
* XPath insures subsets of documents can be retrieved without problems. 
* XSLT insures elements can be rewritten from XML input to any output. 

*** Thus XML, XPath, and XSLT becomes a powerful generic data manipulation pipeline. *** But more importantly, it allows us to transform the elements we wish to extract into an HTML table, while ignoring all other elements. We are now able to do all of this in very few lines of code \cite{Behnel2018}.

In [11]:
import lxml
import lxml.etree
"""
1. find the root
2. match on all `dl` tags and begin producing a `table` of them
3. match on all `dd` and `dt` tags and 
  * place them into their own rows (<tr> tags)
    * inside their own cells (<td> tags)
"""
xslt_root = lxml.etree.XML('''\
 <xsl:stylesheet version="1.0" xmlns:xsl="http://www.w3.org/1999/XSL/Transform">
     <xsl:template match="/">
        <html><body>
             <xsl:for-each select=".//dl">
                 <table>
                     <tr>
                         <xsl:for-each select=".//dd">
                         <td>
                             <xsl:value-of select="." />
                         </td>
                         </xsl:for-each>
                     </tr>
                     <tr>
                         <xsl:for-each select=".//dt">
                             <td>
                                 <xsl:value-of select="." />
                             </td>
                         </xsl:for-each>
                     </tr>
                 </table>
             </xsl:for-each>
         </body></html>
     </xsl:template>
 </xsl:stylesheet>''')

# compile it from the XSLT
transform = lxml.etree.XSLT(xslt_root)

# apply the XSLT transformation to an HTML compiled representation of requested page
tree_to_scrape = transform(lxml.etree.HTML(r.content))

## Return string of output. Uncomment to view output
# str(tree_to_scrape)

None

In [9]:
"""
1. convert to string
2. read with pandas
3. grab the first/zeroth table
4. transpose it
5. switch indexing order for columns (optional)
""" 
firstTable = pd.read_html(str(tree_to_scrape))[0].T[[1,0]]

# return pandas `DataFrame` 
firstTable

,1,0
0,order,Sequential numbering of rows (1 through 9159)
1,pacl_country,String country identifier.
2,year,Calendar year
...,...,...
68,agedem,Age in years of the current regime as classifi...
69,agereg,Age in years of the current regime as classifi...
70,stra,Sum of past transitions to authoritarianism in...


### JSON


**JavaScript Object Notation (JSON)** has become one of the standard formats for sending data by HTTP request between web browsers and other applications. It is a much more flexible data format than a tabular text form like CSV, and much smaller in size than XML files. JSON is very nearly valid Python code. 

To convert a JSON string to Python form, use `json.loads`. `json.dumps` on the other hand converts a Python object back to JSON. Because the JSON structure is unpredictable, how you convert a JSON object or list of objects to a DataFrame or some other data structure for analysis will be up to you. Conveniently, we are able to pass a list of JSON objects to the `DataFrame` constructor and select subsets of fields.


In [10]:
import requests

# Endpoint for retrieving Yahoo Weather for Toronto, ON. 
apiEndpoint =  'https://query.yahooapis.com/v1/public/yql?q=select%20*%20from%20weather.forecast%20where%20woeid%20in%20(select%20woeid%20from%20geo.places(1)%20where%20text%3D%22toronto%2C%20on%22)&format=json&env=store%3A%2F%2Fdatatables.org%2Falltableswithkeys'


response_json = pd.read_json(apiEndpoint, lines=False)
response_json

,query
count,1
created,2018-08-14T20:17:08Z
lang,en-US
results,"{'channel': {'units': {'distance': 'mi', 'pres..."


How do we get a pandas `DataFrame` from the results and not the response? `results` actually contains a proper python object. So, it can be manually manipulated or we can reapply pandas to `results`.

And, this can be repeated. The best way to repeat would be to apply `map()` and then combine horizontally or vertically depending on the situation. Moreover, dummy variables per expansion would allow checking a field's presence. Additionally, it would be preferable to wrap these in a python function to compartmentalize the code used for data manipulation.

In [11]:
pd.DataFrame(response_json["query"]["results"])

,channel
astronomy,"{'sunrise': '6:21 am', 'sunset': '8:23 pm'}"
atmosphere,"{'humidity': '70', 'pressure': '999.0', 'risin..."
description,"Yahoo! Weather for Toronto, ON, CA"
...,...
ttl,60
units,"{'distance': 'mi', 'pressure': 'in', 'speed': ..."
wind,"{'chill': '77', 'direction': '185', 'speed': '..."


## Exercise 1

People are always being elected and unelected. It can be hard to keep up. Why not write a web scraper?
Extract the *Members of Parliament* in the [Canadian House of Commons](https://www.ourcommons.ca/) (House of Commons, 2018) 

In [12]:
## Your Code Here
None

In [13]:
### SOLUTION
# If they inspect the site and don't jump the gun, they'll discover the the Gov. Open Data intiatives are alive and well.
r = requests.get('https://www.ourcommons.ca/Parliamentarians/en/members/export?output=CSV&listSeperator=,')
pd.read_csv(filepath_or_buffer=io.StringIO(r.text))

,Honorific Title,First Name,Last Name,Constituency,Province / Territory,Political Affiliation,Start Date,End Date,Unnamed: 8
0,NaN,Ziad,Aboultaif,Edmonton Manning,Alberta,Conservative,10/19/2015 12:00:00 AM,NaN,NaN
1,NaN,Dan,Albas,Central OkanaganSimilkameenNicola,British Columbia,Conservative,10/19/2015 12:00:00 AM,NaN,NaN
2,NaN,Harold,Albrecht,KitchenerConestoga,Ontario,Conservative,10/19/2015 12:00:00 AM,NaN,NaN
...,...,...,...,...,...,...,...,...,...
332,NaN,David,Yurdiga,Fort McMurrayCold Lake,Alberta,Conservative,10/19/2015 12:00:00 AM,NaN,NaN
333,NaN,Salma,Zahid,Scarborough Centre,Ontario,Liberal,10/19/2015 12:00:00 AM,NaN,NaN
334,NaN,Bob,Zimmer,Prince GeorgePeace RiverNorthern Rockies,British Columbia,Conservative,10/19/2015 12:00:00 AM,NaN,NaN


## Scraping the Easy Way

There is of course nothing wrong with a more interactive or manual approach for webscraping using more interactive tools. We only focus on automating the process for repetition. But it should be noted that websites are not static and will often change. For example, people profiles for Members of Parliament change on a regular basis. 


Below are some alternatives for scraping

* [Google Chrome Web Scraper extension](http://webscraper.io/)
* [import.io](http://import.io)
* [Spooky Stuff](https://github.com/tribbloid/spookystuff)

### Python Libraries for Web Scraping
There are many libraries that can be used for scraping web resources. We chose to use the `requests` library largely due to its ability to fully interact with Web APIs and its ability to reduce most use-cases to one-line of code in comparison to other libraries. Similarly, we make use of `lxml` due to its support for XML, XPath, and XSLT for easier transformation into a format immediately useable by `pandas`.Below are the libraries in Python that one would find relevant for the web scraping domain.

* [urllib2](https://docs.python.org/3.6/library/urllib.request.html)
* [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
  * **NOTE**: `lxml` is used by the library.
* [scrapy](https://docs.scrapy.org/en/latest/)
  * **NOTE**: library is for crawling entire websites, and not  requesting only one specific Web Resource.
* [requests](http://docs.python-requests.org/en/master/)
* [RoboBrowser](https://robobrowser.readthedocs.io/en/latest/readme.html)
  * For scraping and filling out forms automatically. Does not render dynamic pages.
* [lxml](http://lxml.de/)

One topic we don't cover are headless web browser, which simulate an actual browser, enabling scraping from the *Document Object Model*, allowing for web scraping of dynamic web resources.  The following are libraries and tools specifically for addressing this use-case.

* [splash](https://splash.readthedocs.io/en/stable/)
* [selenium](https://www.seleniumhq.org/)
* [Splinter](https://splinter.readthedocs.io/en/latest/)
* [phantompy](https://phantompy.readthedocs.io/en/latest/)
  * [phantomjs](http://phantomjs.org/)
  
## End of Part 2

This notebook makes up one part of this module. Now that you have completed this part, please proceed to the next notebook in this module.
If you have any questions, please reach out to your peers using the discussion boards. If you and your peers are unable to come to a suitable conclusion, do not hesitate to reach out to your instructor on the designated discussion board.

# Links

* http://blog.miguelgrinberg.com/post/easy-web-scraping-with-python
  * About scraping using other python libraries, as well as crawling entire websites.
* http://scrapy.org/
  * About writing scrapers as configeration files via scrapy.
* https://docs.python.org/2/library/urllib2.html
  * Documentation for urlib2 library
* http://docs.python-requests.org/en/latest/
* The Absolute Minimum Every Software Developer Absolutely, Positively Must Know About Unicode and Character Sets (No Excuses!)
  * https://www.joelonsoftware.com/2003/10/08/the-absolute-minimum-every-software-developer-absolutely-positively-must-know-about-unicode-and-character-sets-no-excuses/
* http://import.io
  * A web-based platform for extracting data from websites without writing any code.
* http://www.crummy.com/software/BeautifulSoup/
  * Popular alternative to lxml for web/screen scraping
* http://pbpython.com/web-scraping-mn-budget.html
  * Tutorial using BeautifulSoup with requests library, pandas, numpy and mathplotlib
* Python Regular Expressions Cheat Sheet
  * https://pycon2016.regex.training/cheat-sheet

# References

Behnel, S. et al, (2018). Xpath and xslt with lxml [online](http://lxml.de/xpathxslt.html)

House of Commons, (2018). Members of Parliament. Retrieved Aug 21, 2018 from http://www.ourcommons.ca/Parliamentarians/en/members

McKinney, W. (2017). Python for data analysis: Data wrangling with Pandas, NumPy, and IPython (2nd Ed.). O'Reilly Media.